In [ ]:
input_data_path = '/home/ernohanninen/master_project/work/3c/9624ea1129b500b8af2d31f74caffc/scanvi_adata.h5ad'  #Input non-normalizad data
batch = 'batch'
labels = 'label'
epochs = None

In [ ]:
print(epochs)
print(type(epochs))
if epochs == "None":
    epochs = None
else:
    epochs = int(epochs)
    
print(epochs)
print(type(epochs))

In [ ]:
import scanpy as sc
import scib
import pandas as pd
import numpy as np
from scvi.model.utils import mde
import pymde

In [ ]:

adata = sc.read(input_data_path)
hvg  = adata.var.index
print(adata)
print(hvg)
print(type(hvg))
print(len(hvg))

In [ ]:
#adata_scanvi = scIB.integration.scanvi(adata, batch_key=batch, hvg=hvgList,labels=labels, max_epochs=epochs)
#adata_scanvi

In [ ]:
#sc.pp.neighbors(adata_scanvi, use_rep="X_emb") #adata.obsm[X_emb] contains the output of get_latent_representation -function
#sc.tl.leiden(adata_scanvi)

In [ ]:
#Visualize scVI's learned embeddings. Using pymnde package, an alternative to UMAP, that is GPU-accelerated.
#adata_scanvi.obsm["X_mde"] = mde(adata_scanvi.obsm["X_emb"])

In [ ]:
#sc.pl.embedding(adata_scanvi, basis="X_mde", color=["batch", "leiden"], frameon=False, ncols=1)

In [ ]:
#sc.tl.umap(adata_scanvi)
#sc.pl.umap(adata_scanvi, color=['batch', 'cell_type'], frameon=False, wspace=0.6)